In [ ]:
from dask.distributed import Client, LocalCluster
from dask import delayed, dataframe as dd
import dask
import pandas as pd
import geopandas as gpd
import math
import numpy as np
import momepy
import matplotlib.pyplot as plt
import contextily as ctx
from math import ceil
from shapely.geometry import box
from tqdm import tqdm

In [ ]:
local_crs = 27700
place = "test"
lat = 55.86421405612109
lng = -4.251846930489373
country = "UK"
crs=4326
radius=1

In [ ]:
streets = gpd.read_parquet(f"../output/{place}/streets_raw.pq").explode().to_crs(local_crs).reset_index(drop=True)

buildings = gpd.read_parquet(f"../output/{place}/buildings_raw.pq").to_crs(local_crs)

study_area = gpd.read_parquet(f"../output/{place}/study_area.pq").to_crs(local_crs)

water = gpd.read_parquet(f"../output/{place}/water.pq").to_crs(local_crs)

rail = gpd.read_parquet(f"../output/{place}/rail_raw.pq").to_crs(local_crs).reset_index(drop=True)

In [ ]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='70GB')

client = Client(daskCluster)

client

In [ ]:
streets[~streets['geometry'].is_valid]

In [ ]:
buildings[~buildings['geometry'].is_valid]

In [ ]:
water[~water['geometry'].is_valid]

In [ ]:
buildings.plot()

In [ ]:
# buildings = momepy.preprocess(buildings.reset_index(), size=40,
#                               compactness=0.2, islands=True)

In [ ]:
buildings

In [ ]:
# Check for invalid geometries
invalid_geometries = buildings[~buildings.geometry.is_valid]

# If there are invalid geometries, attempt to fix them
if not invalid_geometries.empty:
    print(f"Found {len(invalid_geometries)} invalid geometries. Attempting to fix...")
    buildings.geometry = buildings.geometry.buffer(0)

    # Recheck for invalid geometries
    still_invalid = buildings[~buildings.geometry.is_valid]
    if still_invalid.empty:
        print("All invalid geometries fixed.")
    else:
        print(f"Could not fix {len(still_invalid)} geometries.")
else:
    print("No invalid geometries found.")

In [ ]:
buildings['uID'] = momepy.unique_id(buildings)

In [ ]:
water_bodies_boundaries = []

# Explode the GeoDataFrame into a GeoSeries of polygons and multipolygons
gs = water.explode()

# Convert each polygon into a MultiLineString
mls = gs.geometry.boundary

# Convert the MultiLineString into a DataFrame
water_bodies = gpd.GeoDataFrame({'geometry': mls})

for water_body in water_bodies.geometry:
    # Create a LineString object from the coordinates
    water_bodies_boundaries.append(water_body)

# # Create a GeoDataFrame of the water area boundaries
water_bodies_boundaries = gpd.GeoDataFrame({'geometry': water_bodies_boundaries}).reset_index(drop=True)

In [ ]:
enclosures = momepy.enclosures(streets, limit=study_area, additional_barriers=[water_bodies_boundaries, rail])

In [ ]:
num_chunks = ceil(len(buildings) / 1400000)

In [ ]:
def split_bounds(bounds, num_chunks):
    minx, miny, maxx, maxy = bounds
    width = maxx - minx
    height = maxy - miny

    # Assuming a square grid for simplicity
    chunks_per_side = math.ceil(math.sqrt(num_chunks))
    chunk_width = width / chunks_per_side
    chunk_height = height / chunks_per_side

    chunks = []
    for i in range(chunks_per_side):
        for j in range(chunks_per_side):
            new_minx = minx + i * chunk_width
            new_miny = miny + j * chunk_height
            new_maxx = new_minx + chunk_width
            new_maxy = new_miny + chunk_height

            # Create a new bounding box (as a shapely box) for each chunk
            chunk_bounds = box(new_minx, new_miny, new_maxx, new_maxy)
            chunks.append(chunk_bounds)

    return gpd.GeoSeries(chunks)

# Example usage
chunks = split_bounds(enclosures.total_bounds, num_chunks)

In [ ]:
chunks = gpd.GeoDataFrame(geometry = chunks)
chunks["chunk_ID"] = range(len(chunks))
enclosures["centroid"] = enclosures.centroid
enclosures = enclosures.sjoin(chunks, how="left")
enclosures = enclosures[["geometry", "chunk_ID", "eID"]]

In [ ]:
chunks

In [ ]:
# Plotting each chunk
fig, ax = plt.subplots()
chunks.plot(ax=ax, edgecolor='black', facecolor='none')
enclosures.plot(ax=ax, column='chunk_ID', categorical=True, legend=True)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=buildings.crs)
plt.show()

In [ ]:
buildings = buildings[["geometry"]].sjoin(enclosures, how="left")

In [ ]:
buildings = buildings.dropna()[["geometry", "chunk_ID"]]
buildings["uID"] = range(len(buildings))
old_buildings = buildings
buildings = buildings.set_geometry('geometry')

In [ ]:
buildings = buildings.to_crs(local_crs)
enclosures = enclosures.to_crs(local_crs)

In [ ]:
tessellation = gpd.GeoDataFrame()

for index, chunk in tqdm(chunks.iterrows(), total= chunks.shape[0]):
    chunk_tessellation = momepy.Tessellation(buildings[buildings["chunk_ID"]==index], unique_id='uID', enclosures=enclosures[enclosures["chunk_ID"] == index], use_dask = True).tessellation
    tessellation = pd.concat([tessellation, chunk_tessellation])
    
tessellation_old = tessellation

In [ ]:
# tessellation.plot()

In [ ]:
tessellation = tessellation_old

In [ ]:
tessellation.to_parquet(f"../output/{place}/tessellation_p1_raw.pq")

In [ ]:
# tessellation = gpd.read_parquet(f"../output/{place}/tessellation_p1_raw.pq")

In [ ]:
tessellation=tessellation.sort_values(by='uID').reset_index(drop=True).dropna(subset=['uID'])

In [ ]:
duplicate_gdf = tessellation[tessellation.duplicated('uID', keep=False)]

In [ ]:
@delayed
def find_valid_from_multiples(gdf, building):
    gdf = gdf[gdf.intersects(building)]
        
    if gdf.shape[0] > 1:
        smallest_area = gdf.geometry.area.idxmin()
        return gdf.loc[[smallest_area]]
    return gdf

# Using a list comprehension for conciseness and efficiency
results = dask.compute([
    find_valid_from_multiples(duplicate_gdf[duplicate_gdf['uID'] == uid].reset_index(), buildings[buildings["uID"] == uid].geometry.iloc[0])
    for uid in duplicate_gdf["uID"].unique()
])

# Concatenate the results into a single GeoDataFrame
a = gpd.GeoDataFrame(pd.concat(results[0], ignore_index=True))
tessellation = gpd.GeoDataFrame(pd.concat([tessellation, a], ignore_index=True))


In [ ]:
tessellation

In [ ]:
tessellation = tessellation.drop("eID", axis = 1)

In [ ]:
# Convert the 'uID' column of buildings to a set for efficient lookups
uIDs_buildings = set(buildings['uID'])

# Filter the tesellation GeoDataFrame to keep only rows with uID in buildings
tessellation = tessellation[tessellation['uID'].isin(uIDs_buildings)]


In [ ]:
# Convert the 'uID' column of buildings to a set for efficient lookups
uIDs_tessellation = set(tessellation['uID'])

# Filter the tesellation GeoDataFrame to keep only rows with uID in buildings
buildings = buildings[buildings['uID'].isin(uIDs_tessellation)]


In [ ]:
tessellation

In [ ]:
tessellation = tessellation[["uID", "geometry"]]
buildings = buildings[["uID", "geometry"]]

In [ ]:
tessellation[tessellation.duplicated('uID', keep=False)]

In [ ]:
combined = tessellation.merge(buildings, on='uID', how='inner').reset_index()
combined["uID"] = range(len(combined))
combined

In [ ]:
tessellation = gpd.GeoDataFrame(combined[["uID", "geometry_x"]], geometry='geometry_x')
tessellation = tessellation.rename(columns={'geometry_x': 'geometry'})

In [ ]:
tessellation = tessellation.set_geometry("geometry", crs=local_crs)

In [ ]:
buildings = gpd.GeoDataFrame(combined[["uID", "geometry_y"]], geometry='geometry_y')
buildings = buildings.rename(columns={'geometry_y': 'geometry'})

In [ ]:
buildings = buildings.set_geometry("geometry", crs=local_crs)

In [ ]:
tessellation.to_parquet(f"../output/{place}/tessellation_p2.pq")

In [ ]:
buildings.to_parquet(f"../output/{place}/buildings_p2.pq")

In [ ]:
streets.to_parquet(f"../output/{place}/streets_p2.pq")

In [ ]:
# # Create a figure and axis

# # Plot streets in blue on the same axis
# streets.plot(ax=ax, color='blue')

# # Plot buildings in red on the same axis
# buildings.plot(ax=ax, color='red')

# tessellation.plot(ax = ax, color = 'green')

# # Show the plot
# plt.show()